In [2]:
import pandas as pd
import networkx as nx
import geopandas

In [3]:
# To create geometry data, for distance calculation later.
data = pd.read_csv(r"C:\Users\iante\Desktop\Work\Files\imports\busroutes.csv")
data = geopandas.GeoDataFrame(data, geometry=geopandas.points_from_xy(data.Longitude, data.Latitude))
data = data.drop(['RoadName'], axis=1)
data

,ServiceNo,StopSequence,BusStopCode,Latitude,Longitude,geometry
0,10,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
1,10,74,75009,1.354076,103.943391,POINT (103.94339 1.35408)
2,127,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
3,127,22,75009,1.354076,103.943391,POINT (103.94339 1.35408)
4,127A,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
...,...,...,...,...,...,...
25988,NR3,58,45409,1.401898,103.745796,POINT (103.74580 1.40190)
25989,NR3,59,45381,1.400859,103.747128,POINT (103.74713 1.40086)
25990,NR3,61,45479,1.394295,103.746520,POINT (103.74652 1.39430)
25991,NR5,76,22571,1.337804,103.706891,POINT (103.70689 1.33780)


In [4]:
# When read from csv, BusStopCode will become int. So convert to str and zfill 5.
data['BusStopCode'] = data['BusStopCode'].astype(str)
data['BusStopCode'] = data.BusStopCode.str.zfill(5)
data

,ServiceNo,StopSequence,BusStopCode,Latitude,Longitude,geometry
0,10,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
1,10,74,75009,1.354076,103.943391,POINT (103.94339 1.35408)
2,127,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
3,127,22,75009,1.354076,103.943391,POINT (103.94339 1.35408)
4,127A,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
...,...,...,...,...,...,...
25988,NR3,58,45409,1.401898,103.745796,POINT (103.74580 1.40190)
25989,NR3,59,45381,1.400859,103.747128,POINT (103.74713 1.40086)
25990,NR3,61,45479,1.394295,103.746520,POINT (103.74652 1.39430)
25991,NR5,76,22571,1.337804,103.706891,POINT (103.70689 1.33780)


In [5]:
# extract record for node creation.
busstops = data.copy()
busstops = busstops.drop(['StopSequence', 'ServiceNo', 'Latitude', 'Longitude', 'geometry'], axis=1)
tbusstops = [str(row)[2:-2] for row in busstops.values]

In [6]:
# extract information for edge creation.
information = data.copy()
# tinformation = [tuple(row) for row in information.values]
information = information.sort_values(['ServiceNo', 'StopSequence'])
information

,ServiceNo,StopSequence,BusStopCode,Latitude,Longitude,geometry
0,10,1,75009,1.354076,103.943391,POINT (103.94339 1.35408)
803,10,1,16009,1.294253,103.769883,POINT (103.76988 1.29425)
60,10,2,76059,1.352962,103.941652,POINT (103.94165 1.35296)
818,10,2,16089,1.292011,103.769084,POINT (103.76908 1.29201)
78,10,3,76069,1.348753,103.942086,POINT (103.94209 1.34875)
...,...,...,...,...,...,...
20470,NR8,57,43839,1.365273,103.748250,POINT (103.74825 1.36527)
20475,NR8,58,43649,1.365458,103.750602,POINT (103.75060 1.36546)
25696,NR8,59,43589,1.362273,103.751532,POINT (103.75153 1.36227)
25700,NR8,60,43579,1.359117,103.751668,POINT (103.75167 1.35912)


In [15]:
# duplicate to shift stop sequence up by 1
information2 = information.copy()
information2["StopSequence"] = information.StopSequence -1
information2

,ServiceNo,StopSequence,BusStopCode,Latitude,Longitude,geometry
0,10,0,75009,1.354076,103.943391,POINT (103.94339 1.35408)
803,10,0,16009,1.294253,103.769883,POINT (103.76988 1.29425)
60,10,1,76059,1.352962,103.941652,POINT (103.94165 1.35296)
818,10,1,16089,1.292011,103.769084,POINT (103.76908 1.29201)
78,10,2,76069,1.348753,103.942086,POINT (103.94209 1.34875)
...,...,...,...,...,...,...
20470,NR8,56,43839,1.365273,103.748250,POINT (103.74825 1.36527)
20475,NR8,57,43649,1.365458,103.750602,POINT (103.75060 1.36546)
25696,NR8,58,43589,1.362273,103.751532,POINT (103.75153 1.36227)
25700,NR8,59,43579,1.359117,103.751668,POINT (103.75167 1.35912)


In [28]:
joininfo = pd.merge(information, information2, how="inner",
                    on=["ServiceNo", "StopSequence"])

# no good, results =25137. Probably cuts out rows that features buses that run on one-way street
# or occasional error through deduction method.
# e.g. both A->B and B->A buses driving on through the same bus stops.
prev_x = ''
prev_y = ''
prev_Sn = ''
prev_ss = ''
for index, row in joininfo.iterrows():
    if row['BusStopCode_x'] == prev_x and row['ServiceNo'] == prev_Sn and row['StopSequence'] == prev_ss:
        if abs(int(row['BusStopCode_x']) - int(rows['BusStopCode_y'])) > abs(int(row['BusStopCode_x']) - int(prev_y)):
            joininfo.drop(index, inplace=True)
        else:
            joininfo.drop(index-1, inplace=True)
    prev_x, prev_y, prev_Sn, prev_ss = row['BusStopCode_x'], row['BusStopCode_y'], row['ServiceNo'], row['StopSequence']

joininfo

,ServiceNo,StopSequence,BusStopCode_x,Latitude_x,Longitude_x,geometry_x,BusStopCode_y,Latitude_y,Longitude_y,geometry_y
0,10,1,75009,1.354076,103.943391,POINT (103.94339 1.35408),76059,1.352962,103.941652,POINT (103.94165 1.35296)
3,10,1,16009,1.294253,103.769883,POINT (103.76988 1.29425),16089,1.292011,103.769084,POINT (103.76908 1.29201)
4,10,2,76059,1.352962,103.941652,POINT (103.94165 1.35296),76069,1.348753,103.942086,POINT (103.94209 1.34875)
7,10,2,16089,1.292011,103.769084,POINT (103.76908 1.29201),16079,1.291383,103.771298,POINT (103.77130 1.29138)
8,10,3,76069,1.348753,103.942086,POINT (103.94209 1.34875),96289,1.340055,103.948381,POINT (103.94838 1.34005)
...,...,...,...,...,...,...,...,...,...,...
38797,NR8,56,43499,1.362596,103.746867,POINT (103.74687 1.36260),43839,1.365273,103.748250,POINT (103.74825 1.36527)
38798,NR8,57,43839,1.365273,103.748250,POINT (103.74825 1.36527),43649,1.365458,103.750602,POINT (103.75060 1.36546)
38799,NR8,58,43649,1.365458,103.750602,POINT (103.75060 1.36546),43589,1.362273,103.751532,POINT (103.75153 1.36227)
38800,NR8,59,43589,1.362273,103.751532,POINT (103.75153 1.36227),43579,1.359117,103.751668,POINT (103.75167 1.35912)


In [64]:
joininfo = [tuple(row) for row in joininfo.values]

In [ ]:
G=nx.MultiDiGraph()

In [62]:
for busstopcode in tbusstops:
    G.add_node(busstopcode, attr=busstopcode)

5042